In [1]:
import pandas as pd
from os import listdir

In [2]:
dates = []
filelist = []
data_path = 'E:\\Proton Drive\\My files\\September Cryto Data'
for csv in listdir(data_path):
    dates.append(csv.split('.')[0])
    df = pd.read_csv(data_path + '\\' + csv, index_col=False, header=None)
    filelist.append(df)
print(dates)
print(len(filelist))

['2024-09-01', '2024-09-02', '2024-09-03', '2024-09-04', '2024-09-05', '2024-09-06', '2024-09-07', '2024-09-08', '2024-09-09', '2024-09-10', '2024-09-11', '2024-09-12', '2024-09-13', '2024-09-14', '2024-09-15', '2024-09-16', '2024-09-17', '2024-09-18', '2024-09-19', '2024-09-20', '2024-09-21', '2024-09-22', '2024-09-23', '2024-09-24', '2024-09-25', '2024-09-26', '2024-09-27', '2024-09-28', '2024-09-29', '2024-09-30']
30


In [3]:
header = ['ticker', 'name', 'price', 'mc', '24hrvolume', 'supply', 'date']
for i in range(0, 30):
    filelist[i]['date'] = dates[i]

for df in filelist:
    df.columns = header
    print(df)

     ticker            name         price         mc 24hrvolume   supply  \
0       BTC         Bitcoin      $ 58,204    $ 1.15T  $ 103.64B   19.75M   
1       ETH        Ethereum    $ 2,476.63  $ 297.94B   $ 47.68B  120.30M   
2      USDT          Tether    $ 1.001335  $ 118.38B   $ 67.36B  118.23B   
3       BNB    Binance Coin      $ 527.13   $ 77.79B   $ 14.47B  147.58M   
4       SOL          Solana      $ 133.76   $ 62.38B    $ 4.36B  466.34M   
...     ...             ...           ...        ...        ...      ...   
9719    ZXE         ZooomEx    $ 0.000107     $ 0.00     $ 0.00     0.00   
9720   ZYRO            Zyro  $ 0.00006831     $ 0.00   $ 45,560     0.00   
9721    ZYX             ZYX    $ 0.006661     $ 0.00     $ 0.00     0.00   
9722    ZZZ     ZZZ.finance    $ 0.700615     $ 0.00     $ 0.00     0.00   
9723  ZZZV2  zzz.finance v2    $ 1.409862     $ 0.00     $ 0.00     0.00   

            date  
0     2024-09-01  
1     2024-09-01  
2     2024-09-01  
3     2024-

In [4]:
def clean_df(df):
    def convert_stringnumbers_to_float(value):
        try:
            # Remove dollar sign and spaces
            value = value.replace('$', '').replace('\u202f', '').replace(',', '').strip()
            
            # Check for the magnitude and convert accordingly
            if value.endswith('T'):
                return float(value[:-1]) * 1e12  # Convert from trillions
            if value.endswith('B'):
                return float(value[:-1]) * 1e9  # Convert from billions
            elif value.endswith('M'):
                return float(value[:-1]) * 1e6  # Convert from millions
            elif value.endswith('K'):
                return float(value[:-1]) * 1e3  # Convert from thousands
            else:
                return float(value)  # No magnitude, convert as is
        except ValueError:
            return None
    
    # Apply the conversion function to the 'Market Cap' column
    df['mc'] = df['mc'].apply(convert_stringnumbers_to_float)
    df['24hrvolume'] = df['24hrvolume'].apply(convert_stringnumbers_to_float)
    df['supply'] = df['supply'].apply(convert_stringnumbers_to_float)
    df['price'] = df['price'].apply(convert_stringnumbers_to_float)
    
    # Convert 'Date' to datetime and create separate columns for year, month, and day
    df['date'] = pd.to_datetime(df['date'])
    df['year'] = df['date'].dt.year
    df['month'] = df['date'].dt.month
    df['day'] = df['date'].dt.day
    
    df_cleaned = df.dropna(subset=['mc', '24hrvolume', 'supply', 'price', 'date'])
    
    print(df.shape)
    print(df_cleaned.shape)
    return df_cleaned

In [5]:
cleaned_dfs = []
for df in filelist:
    cleaned_dfs.append(clean_df(df))

combined_df = pd.concat(cleaned_dfs, ignore_index=True)
combined_df.to_csv('combined_data.csv', index=False)

(9724, 10)
(7888, 10)
(9721, 10)
(7889, 10)
(9721, 10)
(7885, 10)
(9724, 10)
(7889, 10)
(9725, 10)
(7886, 10)
(9725, 10)
(7886, 10)
(9724, 10)
(7885, 10)
(9723, 10)
(7884, 10)
(9723, 10)
(7885, 10)
(9725, 10)
(7883, 10)
(9724, 10)
(7886, 10)
(9724, 10)
(7886, 10)
(9726, 10)
(7887, 10)
(9730, 10)
(7885, 10)
(9731, 10)
(7884, 10)
(9731, 10)
(7877, 10)
(9732, 10)
(7883, 10)
(9734, 10)
(7886, 10)
(9736, 10)
(7884, 10)
(9736, 10)
(7887, 10)
(9736, 10)
(7889, 10)
(9736, 10)
(7888, 10)
(9736, 10)
(7883, 10)
(9737, 10)
(7886, 10)
(9737, 10)
(7885, 10)
(9739, 10)
(7886, 10)
(9739, 10)
(7887, 10)
(9739, 10)
(7885, 10)
(9740, 10)
(7886, 10)
(9740, 10)
(7884, 10)


In [6]:
# Clean so final df only includes tickers which have all their days present
df = pd.read_csv('combined_data.csv')

# Check for duplicate combinations of 'ticker' and 'day'
if df.duplicated(subset=['ticker', 'day']).any():
    print("Duplicates found. Removing duplicates based on first occurrence.")
    # Drop duplicate rows (you can modify this logic to suit your needs)
    df = df.drop_duplicates(subset=['ticker', 'day'])

tickers = df['ticker'].unique()
days = df['day'].unique()

full_index = pd.MultiIndex.from_product([tickers, days], names=['ticker', 'day'])

df_cleaned = df.set_index(['ticker', 'day']).reindex(full_index).reset_index()

df_complete = df_cleaned.dropna(subset=df.columns.difference(['ticker', 'day']), how='any')

df_complete.to_csv('df_complete_cleaned.csv', index=False)

Duplicates found. Removing duplicates based on first occurrence.


In [7]:
import pandas as pd

# Load the cleaned dataframe
df = pd.read_csv('df_complete_cleaned.csv')

# Calculate the average 'mc' (market cap) for each ticker
average_mc = df.groupby('ticker')['mc'].mean()

# Sort tickers by average market cap in descending order and select the top 100
top_100_tickers = average_mc.sort_values(ascending=False).head(100).index

# Filter the original dataframe to keep only rows with the top 100 tickers
df_top_100 = df[df['ticker'].isin(top_100_tickers)]

# Save the filtered dataframe to a new CSV file
df_top_100.to_csv('df_top_100_tickers.csv', index=False)